In [76]:
import pandas as pd
import numpy as np
import geopandas as gpd
import rasterio

import os, h5py, sys
from pathlib import Path
import openpyxl

import holoviews as hv

In [77]:
import sys
sys.path.append('../../src/')
from utility_code import RAS_Utility
ras = RAS_Utility()

 ### get_manning_data

In [7]:
# set location of hdf data
paln_hdf_loc = r'../../simulation/WF_WestForkCalcasieu.p14.hdf'
geo_hdf_loc = r'../../simulation/WF_WestForkCalcasieu.g03.hdf'
flow_hdf_loc = r'../../simulation/WF_WestForkCalcasieu.u10.hdf'

In [8]:
ras.get_manning_data(geo_hdf_loc)

,Land Cover Name,Base Manning's n Value,WFCalcRi_1,WFCalcRi_2,WFCalcRi_3,WFCalcRi_4,WFCalcRi_5,WFCalcRi_6,WFCalcRi_7,WFCalcRi_8,...,WFCalcRi_110,WFCalcRi_111,WFCalcRi_112,WFCalcRi_113,WFCalcRi_114,WFCalcRi_115,WFCalcRi_116,WFCalcRi_117,WFCalcRi_118,WFCalcRi_119
0,NoData,0.035,0.04,0.03,0.04,0.03,0.04,0.03,0.04,0.04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,255,0.035,0.04,0.03,0.04,0.03,0.04,0.03,0.04,0.04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Open Water,0.035,0.04,0.03,0.04,0.03,0.04,0.03,0.04,0.04,...,0.2625,0.2625,0.2625,0.2625,0.2625,0.2625,0.2625,0.2625,0.2625,0.2625
3,"Developed, Open Space",0.090,0.04,0.03,0.04,0.03,0.04,0.03,0.04,0.04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Developed, Low Intensity",0.100,0.04,0.03,0.04,0.03,0.04,0.03,0.04,0.04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,"Developed, Medium Intensity",0.100,0.04,0.03,0.04,0.03,0.04,0.03,0.04,0.04,...,0.1200,0.1200,0.1200,0.1200,0.1200,0.1200,0.1200,0.1200,0.1200,0.1200
6,"Developed, High Intensity",0.150,0.04,0.03,0.04,0.03,0.04,0.03,0.04,0.04,...,0.0265,0.0265,0.0265,0.0265,0.0265,0.0265,0.0265,0.0265,0.0265,0.0265
7,Barren Land Rock-Sand-Clay,0.100,0.04,0.03,0.04,0.03,0.04,0.03,0.04,0.04,...,0.0400,0.0400,0.0400,0.0400,0.0400,0.0400,0.0400,0.0400,0.0400,0.0400
8,Deciduous Forest,0.120,0.04,0.03,0.04,0.03,0.04,0.03,0.04,0.04,...,0.0375,0.0375,0.0375,0.0375,0.0375,0.0375,0.0375,0.0375,0.0375,0.0375
9,Evergreen Forest,0.120,0.04,0.03,0.04,0.03,0.04,0.03,0.04,0.04,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


 ### get_model_info
 
 This function needs to take the typical RAS folder structure and return the relevant plan associations for each hec ras plan

In [78]:
%cd ../../simulation

[WinError 2] The system cannot find the file specified: '../../simulation'
c:\Users\christianl\repos\lwi-apps\simulation


In [79]:
%pwd

'c:\\Users\\christianl\\repos\\lwi-apps\\simulation'

In [80]:
path = %pwd
path=Path(path)

In [81]:
path

WindowsPath('c:/Users/christianl/repos/lwi-apps/simulation')

In [104]:
for child in Path('.').glob('*.prj'):
    if child.is_file():
        prj_file=child

prj_file

WindowsPath('WF_WestForkCalcasieu.prj')

In [95]:
prj_file_list=(Path.read_text(prj_file).split("\n"))
plan_list = [f'.{x.split("=")[-1]}' for x in prj_file_list if x.startswith("Plan File=")]
plan_list 

['.p01',
 '.p02',
 '.p03',
 '.p04',
 '.p05',
 '.p06',
 '.p07',
 '.p08',
 '.p09',
 '.p10',
 '.p11',
 '.p12',
 '.p13',
 '.p14',
 '.p15',
 '.p16',
 '.p17',
 '.p18',
 '.p19',
 '.p20',
 '.p21',
 '.p22',
 '.p23',
 '.p24']

In [102]:
# find all files whose extension is in plan_list return the full path
plan_file_list = [x for x in Path('.').glob('*') if x.suffix in plan_list]
plan_file_list

[WindowsPath('Backup.p01'),
 WindowsPath('WF_WestForkCalcasieu.p01'),
 WindowsPath('WF_WestForkCalcasieu.p02'),
 WindowsPath('WF_WestForkCalcasieu.p03'),
 WindowsPath('WF_WestForkCalcasieu.p04'),
 WindowsPath('WF_WestForkCalcasieu.p05'),
 WindowsPath('WF_WestForkCalcasieu.p06'),
 WindowsPath('WF_WestForkCalcasieu.p07'),
 WindowsPath('WF_WestForkCalcasieu.p08'),
 WindowsPath('WF_WestForkCalcasieu.p09'),
 WindowsPath('WF_WestForkCalcasieu.p10'),
 WindowsPath('WF_WestForkCalcasieu.p11'),
 WindowsPath('WF_WestForkCalcasieu.p12'),
 WindowsPath('WF_WestForkCalcasieu.p13'),
 WindowsPath('WF_WestForkCalcasieu.p14'),
 WindowsPath('WF_WestForkCalcasieu.p15'),
 WindowsPath('WF_WestForkCalcasieu.p16'),
 WindowsPath('WF_WestForkCalcasieu.p17'),
 WindowsPath('WF_WestForkCalcasieu.p18'),
 WindowsPath('WF_WestForkCalcasieu.p19'),
 WindowsPath('WF_WestForkCalcasieu.p20'),
 WindowsPath('WF_WestForkCalcasieu.p21'),
 WindowsPath('WF_WestForkCalcasieu.p22'),
 WindowsPath('WF_WestForkCalcasieu.p23'),
 Windo

In [170]:
# create a dictionary of plan names and file paths the name is found in the plan file
plan_dict = {}
for plan_file in plan_file_list:
    plan_name = Path.read_text(plan_file).split("\n")[0].split("=")[-1]
    plan_dict[plan_name] = plan_file

# create a dataframe of plan names and file paths
plan_df = pd.DataFrame.from_dict(plan_dict, orient='index', columns=['plan_file'])
plan_df.head()

,plan_file
,Backup.p01
Laura 2020,WF_WestForkCalcasieu.p01
002YR,WF_WestForkCalcasieu.p02
100YR,WF_WestForkCalcasieu.p03
CTRL01_Laura 2020_A,WF_WestForkCalcasieu.p04


In [171]:
# Path.read_text(plan_dict['002YR']).split("\n")

In [172]:
# add the plans geometry file to plan_df
plan_df['geom_file'] = plan_df['plan_file'].apply(lambda x: Path.read_text(x).split("\n")[4].split("=")[-1])
plan_df.head()

,plan_file,geom_file
,Backup.p01,g08
Laura 2020,WF_WestForkCalcasieu.p01,g01
002YR,WF_WestForkCalcasieu.p02,g01
100YR,WF_WestForkCalcasieu.p03,g02
CTRL01_Laura 2020_A,WF_WestForkCalcasieu.p04,g03


In [175]:
# use glob to find the geometry file
plan_df['geom_file'] = plan_df['geom_file'].apply(lambda x: [y for y in Path('.').glob(f'*{x}')][0])
plan_df.head()

,plan_file,geom_file
,Backup.p01,WF_WestForkCalcasieu.g08
Laura 2020,WF_WestForkCalcasieu.p01,Backup.g01
002YR,WF_WestForkCalcasieu.p02,Backup.g01
100YR,WF_WestForkCalcasieu.p03,WF_WestForkCalcasieu.g02
CTRL01_Laura 2020_A,WF_WestForkCalcasieu.p04,WF_WestForkCalcasieu.g03


In [180]:
for child in Path('.').glob('*.hdf'):
    if child.is_file():
        print(child)

Backup.u01.hdf
WestForkCalcasieu_2.g11.hdf
WestForkCalcasieu_2.p11.hdf
WestForkCalcasieu_2.p12.tmp.hdf
WestForkCalcasieu_2.p24.hdf
WestForkCalcasieu_2.p24.tmp.hdf
WF_WestForkCalcasieu.g01.hdf
WF_WestForkCalcasieu.g02.hdf
WF_WestForkCalcasieu.g02.tmp.hdf
WF_WestForkCalcasieu.g03.hdf
WF_WestForkCalcasieu.g04.hdf
WF_WestForkCalcasieu.g05.hdf
WF_WestForkCalcasieu.g06.hdf
WF_WestForkCalcasieu.g07.hdf
WF_WestForkCalcasieu.g08.hdf
WF_WestForkCalcasieu.g09.hdf
WF_WestForkCalcasieu.p01.hdf
WF_WestForkCalcasieu.p01.tmp.hdf
WF_WestForkCalcasieu.p02.hdf
WF_WestForkCalcasieu.p03.hdf
WF_WestForkCalcasieu.p03.tmp.hdf
WF_WestForkCalcasieu.p04.hdf
WF_WestForkCalcasieu.p05.hdf
WF_WestForkCalcasieu.p06.hdf
WF_WestForkCalcasieu.p07.hdf
WF_WestForkCalcasieu.p08.hdf
WF_WestForkCalcasieu.p08.tmp.hdf
WF_WestForkCalcasieu.p09.hdf
WF_WestForkCalcasieu.p14.hdf
WF_WestForkCalcasieu.p15.hdf
WF_WestForkCalcasieu.p16.hdf
WF_WestForkCalcasieu.p17.hdf
WF_WestForkCalcasieu.p18.hdf
WF_WestForkCalcasieu.p19.hdf
WF_WestFo